# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [43]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
df.loc[:, df.dtypes == object]

,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,RENT,Source Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
1,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
2,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
3,RENT,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
4,MORTGAGE,Not Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
...,...,...,...,...,...,...,...,...,...,...
68812,RENT,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68813,RENT,Not Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68814,MORTGAGE,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68815,MORTGAGE,Verified,Jan-2019,low_risk,n,f,May-2019,Individual,N,N


In [6]:
df_numbered = df.copy()
label_encoder = LabelEncoder()

In [7]:
label_encoder.fit(df_numbered["home_ownership"])
df_numbered["home_ownership_#"] = label_encoder.transform(df_numbered["home_ownership"])

In [8]:
label_encoder.fit(df_numbered["verification_status"])
df_numbered["verification_status_#"] = label_encoder.transform(df_numbered["verification_status"])

In [9]:
label_encoder.fit(df_numbered["issue_d"])
df_numbered["issue_d_#"] = label_encoder.transform(df_numbered["issue_d"])

In [10]:
label_encoder.fit(df_numbered["loan_status"])
df_numbered["loan_status_#"] = label_encoder.transform(df_numbered["loan_status"])

In [11]:
label_encoder.fit(df_numbered["pymnt_plan"])
df_numbered["pymnt_plan_#"] = label_encoder.transform(df_numbered["pymnt_plan"])

In [12]:
label_encoder.fit(df_numbered["initial_list_status"])
df_numbered["initial_list_status_#"] = label_encoder.transform(df_numbered["initial_list_status"])

In [13]:
label_encoder.fit(df_numbered["next_pymnt_d"])
df_numbered["next_pymnt_d_#"] = label_encoder.transform(df_numbered["next_pymnt_d"])

In [14]:
label_encoder.fit(df_numbered["application_type"])
df_numbered["application_type_#"] = label_encoder.transform(df_numbered["application_type"])

In [15]:
label_encoder.fit(df_numbered["hardship_flag"])
df_numbered["hardship_flag_#"] = label_encoder.transform(df_numbered["hardship_flag"])

In [16]:
label_encoder.fit(df_numbered["debt_settlement_flag"])
df_numbered["debt_settlement_flag_#"] = label_encoder.transform(df_numbered["debt_settlement_flag"])

In [17]:
df_numbered.drop(columns = ["home_ownership", 
                            "verification_status", 
                            "issue_d", 
                            "loan_status", 
                            "pymnt_plan", 
                            "initial_list_status",
                            "next_pymnt_d",
                            "application_type",
                            "hardship_flag",
                            "debt_settlement_flag"], inplace = True )
df_numbered.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_#,verification_status_#,issue_d_#,loan_status_#,pymnt_plan_#,initial_list_status_#,next_pymnt_d_#,application_type_#,hardship_flag_#,debt_settlement_flag_#
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,3,1,2,1,0,1,1,0,0,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,2,2,1,0,1,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,2,2,1,0,1,1,0,0,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,3,2,2,1,0,1,1,0,0,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,2,1,0,1,1,0,0,0


## Split the Data into Training and Testing

In [18]:
# Create our features
X = df_numbered.copy()
X.drop("loan_status_#", axis=1, inplace=True)
X.head()

# Create our target
y = df_numbered["loan_status_#"]

In [19]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,home_ownership_#,verification_status_#,issue_d_#,pymnt_plan_#,initial_list_status_#,next_pymnt_d_#,application_type_#,hardship_flag_#,debt_settlement_flag_#
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,5.572240e+04,1.812779,0.669994,0.805542,0.0,0.876121,0.616839,0.139660,0.0,0.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,5.095845e+04,0.941313,0.719105,0.714932,0.0,0.329446,0.486161,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.270000e+02,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,2.288000e+04,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,4.200000e+04,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,7.249900e+04,3.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.426964e+06,3.000000,2.000000,2.000000,0.0,1.000000,1.000000,1.000000,0.0,0.0


In [20]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

1    68470
0      347
Name: loan_status_#, dtype: int64

In [21]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train.shape

(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [22]:
# Create the StandardScaler instance
# YOUR CODE HERE
scaler = StandardScaler()

In [23]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [24]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
from sklearn.ensemble import RandomForestClassifier
# Resample the training data with the BalancedRandomForestClassifier

# YOUR CODE HERE
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)


In [26]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [27]:
# Display the confusion matrix
# YOUR CODE HERE
predictions = rf_model.predict(X_test_scaled)
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,29,63
Actual 1,21,17092


In [28]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00        92
          1       0.99      1.00      0.00      1.00      0.00      0.00     17113

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



In [29]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.10124499636127673, 'last_pymnt_amnt'),
 (0.07211735586230895, 'total_rec_prncp'),
 (0.0634408559389046, 'total_rec_int'),
 (0.06181994230073553, 'total_pymnt_inv'),
 (0.05939836988275112, 'total_pymnt'),
 (0.018118377749007333, 'mo_sin_old_rev_tl_op'),
 (0.017481845706670895, 'bc_open_to_buy'),
 (0.017475022748803214, 'dti'),
 (0.0172104179452492, 'revol_bal'),
 (0.016718304393543704, 'out_prncp'),
 (0.015654975377527792, 'out_prncp_inv'),
 (0.015420682677353326, 'total_bal_il'),
 (0.015392426234514367, 'max_bal_bc'),
 (0.015375354009420576, 'total_rec_late_fee'),
 (0.015346226079273842, 'mo_sin_old_il_acct'),
 (0.015339822214000936, 'tot_hi_cred_lim'),
 (0.015151556606727004, 'total_bal_ex_mort'),
 (0.015060884195635973, 'installment'),
 (0.014414066131646402, 'tot_cur_bal'),
 (0.014136446995457073, 'bc_util'),
 (0.013562445879952215, 'total_bc_limit'),
 (0.01275734063791289, 'mths_since_rcnt_il'),
 (0.0126377461390634, 'il_util'),
 (0.012598261301974738, 'total_rev_hi_lim'),
 (0.

### Easy Ensemble Classifier

In [36]:
# Train the Classifier
# YOUR CODE HERE
from sklearn.ensemble import GradientBoostingClassifier

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

classifier.fit(X_train_scaled, y_train.ravel())
print("Learning rate: ", learning_rate)

print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
print()


Learning rate:  1
Accuracy score (training): 0.994
Accuracy score (validation): 0.993



In [37]:
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=1,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).head(20)

,Prediction,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [40]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.992734670154025


In [41]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,30,62
Actual 1,63,17050


In [46]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.32      0.33      1.00      0.32      0.57      0.30        92
          1       1.00      1.00      0.33      1.00      0.57      0.35     17113

avg / total       0.99      0.99      0.33      0.99      0.57      0.35     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Gradient Boosting Classifier had the highest accuracy score of 0.992734670154025.

2. Which model had the best recall score?

   Both models had a recall score of .99.

3. Which model had the best geometric mean score?

    The Gradient Boosting Classifier had the geometric mean score of 0.57.

4. What are the top three features?

    The top 3 features are last payment amount, total recieved principle, and total received interest.